In [1]:
# 📊 Data Handling
import pandas as pd
import numpy as np

# 📈 Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ⚙️ Preprocessing & Scaling
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 📈 Analysis
import scipy.stats as stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
  
# 🧠 Modeling
from sklearn.cluster import KMeans, DBSCAN
import hdbscan
from sklearn.metrics import silhouette_score

 # Part 1: Basic Information

SQL Query:
<br>
-- Step 1: همه آدرس‌های قرارداد
WITH contract_addresses AS (
  SELECT DISTINCT to_address AS address
  FROM `bigquery-public-data.crypto_ethereum.traces`
  WHERE call_type = 'create' AND to_address IS NOT NULL
),

<br> -- Step 2: آدرس‌هایی که تراکنش داشتند و تاریخچه‌شون
active_wallets AS (
  SELECT
    from_address AS address,
    COUNT(*) AS tx_count,
    MIN(block_timestamp) AS first_tx,
    MAX(block_timestamp) AS last_tx
  FROM `bigquery-public-data.crypto_ethereum.transactions`
  WHERE block_timestamp >= TIMESTAMP("2025-04-01")
  GROUP BY from_address
)

<br> -- Step 3: فقط EOAها رو نگه می‌داریم و به موجودی وصله می‌زنیم
SELECT
  a.address,
  a.tx_count,
  a.first_tx,
  a.last_tx,
  b.eth_balance
FROM
  active_wallets a
LEFT JOIN
  contract_addresses c
ON
  a.address = c.address
JOIN
  `bigquery-public-data.crypto_ethereum.balances` b
ON
  a.address = b.address
WHERE
  c.address IS NULL -- فقط EOAها
ORDER BY
  tx_count DESC;

In [2]:
df1 = pd.read_csv('/Users/bahareh/Desktop/Data/part 1.csv')

df1['first_tx'] = pd.to_datetime(df1['first_tx'])
df1['last_tx'] = pd.to_datetime(df1['last_tx'])

df1['eth_balance'] = pd.to_numeric(df1['eth_balance'], errors='coerce')
df1['eth'] =round(df1['eth_balance'] / 1e18)
df1 = df1.rename(columns={'address': 'wallet_address'})

df1 = df1.drop(['eth_balance'],axis=1)
df1.head()

,wallet_address,tx_count,first_tx,last_tx,eth
0,0x1a1c87d9a6f55d3bbb064bff1059ad37b6bdc097,580800,2025-04-01 00:00:59+00:00,2025-05-03 17:40:11+00:00,2.0
1,0x974caa59e49682cda0ad2bbe82983419a2ecc400,312795,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,10204.0
2,0x46340b20830761efd32832a74d7169b29feb9758,302996,2025-04-01 00:00:35+00:00,2025-05-03 17:46:11+00:00,19402.0
3,0xf70da97812cb96acdf810712aa562db8dfa3dbef,275100,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,368.0
4,0x264bd8291fae1d75db2c5f573b07faa6715997b5,270429,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,3346.0


# Part 2: Transaction Behavior

WITH contracts AS (
  SELECT DISTINCT to_address AS address
  FROM `bigquery-public-data.crypto_ethereum.traces`
  WHERE call_type = 'create' AND to_address IS NOT NULL
),

txs AS (
  SELECT
    block_timestamp,
    from_address,
    to_address,
    value,
    gas,
    gas_price
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    block_timestamp >= TIMESTAMP("2025-03-15")
)

-- فقط تراکنش‌هایی که فرستنده‌شون EOA باشه:
SELECT
  t.*
FROM
  txs t
LEFT JOIN
  contracts c
ON
  t.from_address = c.address
WHERE
  c.address IS NULL

In [5]:
df2 = pd.read_csv('/Users/bahareh/Desktop/Data/part 2.csv')
df2['block_timestamp'] = pd.to_datetime(df2['block_timestamp'])
df2.head()

,block_timestamp,from_address,to_address,value,gas,gas_price
0,2025-05-03 15:47:47+00:00,0x463e5b673d1029989c9b059d36393c539bef9094,0xfce76faf8b213496ca23cea1385cef6b4bd9e940,120083601538014700,21000,1905103281
1,2025-05-03 15:48:11+00:00,0x3ed461af3ea8ed17e7722754c6080d3f0d064bc8,0xdac17f958d2ee523a2206206994597c13d831ec7,0,63209,726666060
2,2025-05-03 15:48:59+00:00,0x7e8242cb6e74e9b82a564877a5fe49c47e3d882b,0x7e8242cb6e74e9b82a564877a5fe49c47e3d882b,0,21000,510000000
3,2025-05-03 15:49:11+00:00,0x3dbdc1c775220bddafd265c05d6e4be7bf4ac81f,0xa5c8b1c32d05df6f4e6267485452f784e8ae5007,0,86988,890363963
4,2025-05-03 15:49:11+00:00,0xc35fb86f962ea955751a793a007b5cdd44f798d7,0xdac17f958d2ee523a2206206994597c13d831ec7,0,65116,495663673


In [6]:
wallet_stats = []

for address, group in df2.groupby('from_address'):
    group = group.sort_values('block_timestamp')
    group['value'] = pd.to_numeric(group['value'], errors='coerce')
    
    total_tx = len(group)
    avg_value = round(group['value'].mean()/ 1e18)
    
    if total_tx > 1:
        time_diffs = group['block_timestamp'].diff().dt.total_seconds() / 86400 
        avg_time_gap = avg_time_gap = time_diffs.mean() if total_tx > 1 else 0
    else:
        avg_time_gap = None

    min_date = group['block_timestamp'].min()
    max_date = group['block_timestamp'].max()
    total_days = (max_date - min_date).days + 1 
    weeks = total_days / 7 if total_days > 0 else 1
    months = total_days / 30 if total_days > 0 else 1

    tx_per_day = total_tx / total_days if total_days > 0 else total_tx
    tx_per_week = total_tx / weeks
    tx_per_month = total_tx / months

    wallet_stats.append({
        'wallet_address': address,
        'total_tx': total_tx,
        'avg_tx_value': avg_value,
        'tx_per_day': tx_per_day,
        'tx_per_week': tx_per_week,
        'tx_per_month': tx_per_month,
        'avg_time_gap_days': avg_time_gap
    })

result_df = pd.DataFrame(wallet_stats)

eth_price_usd = 3500

result_df['avg_value_usd'] = result_df['avg_tx_value'] * eth_price_usd

result_df['avg_time_gap_days'] = result_df['avg_time_gap_days'].apply(
    lambda x: 0 if pd.isna(x) or x < 1 else int(x)
)

result_df

,wallet_address,total_tx,avg_tx_value,tx_per_day,tx_per_week,tx_per_month,avg_time_gap_days,avg_value_usd
0,0x00000000000030e5959659622cb7eb50aa20ee52,5,0,0.104167,0.729167,3.125000,11,0
1,0x00000000000067f5b5b06c66a9a8d4445ba98083,12,0,0.244898,1.714286,7.346939,4,0
2,0x000000000000c8dcb423f2095460086e0b381044,5,0,0.227273,1.590909,6.818182,5,0
3,0x00000000000124d994209fbb955e0217b5c2eca1,17,0,0.346939,2.428571,10.408163,3,0
4,0x00000000000947821264914ad2c75f871aa2d026,32,0,0.653061,4.571429,19.591837,1,0
...,...,...,...,...,...,...,...,...
36294,0xfff7265cbac1d028a0e4029ba311ebd4f79573d3,1,0,1.000000,7.000000,30.000000,0,0
36295,0xfff7953ed69781fb8040a37b63773a9349bbd711,1,0,1.000000,7.000000,30.000000,0,0
36296,0xfff874445cdbc96743924a7ccad12f5d7f332d77,1,0,1.000000,7.000000,30.000000,0,0
36297,0xfff9326b779fb266a3ce48322d389f94312a18f2,1,0,1.000000,7.000000,30.000000,0,0


# Final Data

In [7]:
df_final = pd.merge(df1,result_df, on='wallet_address',how='inner')
df_final

,wallet_address,tx_count,first_tx,last_tx,eth,total_tx,avg_tx_value,tx_per_day,tx_per_week,tx_per_month,avg_time_gap_days,avg_value_usd
0,0x1a1c87d9a6f55d3bbb064bff1059ad37b6bdc097,580800,2025-04-01 00:00:59+00:00,2025-05-03 17:40:11+00:00,2.0,393,0,8.020408,56.142857,240.612245,0,0
1,0x974caa59e49682cda0ad2bbe82983419a2ecc400,312795,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,10204.0,396,0,8.250000,57.750000,247.500000,0,0
2,0x46340b20830761efd32832a74d7169b29feb9758,302996,2025-04-01 00:00:35+00:00,2025-05-03 17:46:11+00:00,19402.0,253,1,5.270833,36.895833,158.125000,0,3500
3,0xf70da97812cb96acdf810712aa562db8dfa3dbef,275100,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,368.0,424,0,8.653061,60.571429,259.591837,0,0
4,0x264bd8291fae1d75db2c5f573b07faa6715997b5,270429,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,3346.0,214,0,4.458333,31.208333,133.750000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8893,0x15fba39460a78c168f7acb701ecf43ae19cf458e,37,2025-04-03 03:21:35+00:00,2025-05-03 15:39:11+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0
8894,0x7672c5fa51ea088f5904d66b9f816dcf7ef47cb8,37,2025-04-20 06:14:47+00:00,2025-05-03 16:27:47+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0
8895,0x97d989470a07932579ebbbdf569f130c5f10ec89,37,2025-04-01 02:42:47+00:00,2025-05-02 23:45:11+00:00,0.0,1,1,1.000000,7.000000,30.000000,0,3500
8896,0x0d6fe0bd1d4a27f952ff0f54e849fd7b8678a36f,37,2025-04-06 17:24:11+00:00,2025-05-03 17:28:47+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0


In [8]:
df_final.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/ml-in-crypto/04_Wallet_Canvas/df_final.csv')